In [ ]:
#잠재의미 분석(LSA)
#구조화되지 않은 방대한 문헌 집단에서 주제를 찾아내기 위한 알고리즘
# 맥락과 관련된 단서들을 이용하여 의미를 가진 단어들을 클러스터링, 주제를 추론함
# 통계적 알고리즘

#20가지 주제의 뉴스 데이터
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

In [ ]:
documents[0]

In [ ]:
#뉴스 카테고리
print(dataset.target_names)

In [ ]:
news_df = pd.DataFrame({'document': documents})
#알파벳 이외의 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace('[^a-zA-Z]', ' ')

#길이가 3이하인 단어 제거
news_df['clean_doc'] = news_df['clean_doc'].apply(
    lambda x: ' '.join([w for w in x.split() if len(w) > 3]))

#소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [ ]:
news_df['clean_doc'][0]

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

#토큰화
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())

#불용어 제거
tokenized_doc = tokenized_doc.apply(lambda x : [item for item in x if item not in stop_words])

In [ ]:
print(tokenized_doc[0])

In [ ]:
#tf-idf 행렬을 만들기 위해 다시 역토큰화
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc

In [ ]:
news_df['clean_doc'][0]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#상위 1000개 단어만 처리(max_features)
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape # TF-IDF 행렬의 크기 확인

In [ ]:
from sklearn.decomposition import TruncatedSVD
#행렬 특이값 분해, 11314개의 행을 2-개로 축소, n_components 토픽 수

svd_model = TruncatedSVD(n_components=20)
svd_model.fit(X)
len(svd_model.components_)

In [ ]:
import numpy as np

#토픽 수 x 단어수
np.shape(svd_model.components_)

In [ ]:
svd_model.components_

In [ ]:
#단어 집합, 1000개의 단어
terms = vectorizer.get_feature_names()

#20개의 뉴스그룹별로 추출한 토픽 리스트 출력
def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print('Topic %d:' % (idx+1),
        [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(svd_model.components_,terms)